### Task 1. Imports and loading
Import the packages that you've learned are needed for building linear regression models.

In [16]:
%pip install matplotlib seaborn scikit-learn

  Using cached scikit_learn-1.2.2-cp310-cp310-win_amd64.whl (8.3 MB)
  Using cached scipy-1.10.1-cp310-cp310-win_amd64.whl (42.5 MB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
Note: you may need to restart the kernel to use updated packages.


In [17]:
# Imports
### YOUR CODE HERE ###
# numeric feat.DataFrame
import pandas as pd
import numpy as np
# visualization
import matplotlib.pyplot as plt
import seaborn as sns

# package date conversions for calculating
from datetime import datetime
from datetime import date
from datetime import timedelta

#Packages for OLS, MLR  and Confusion Matrix
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import sklearn.metrics as metrics
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
lr = LinearRegression()

In [19]:
# RUN THIS CELL TO IMPORT YOUR DATA. 
### YOUR CODE HERE ###

# Read in data from NYC TLC dataset provided and load into dataframe
taxi_data = pd.read_csv("2017_Yellow_Taxi_Trip_Data.csv") # index_col parameter specified to avoid "Unnamed: 0" column when reading in data from csv

# Display "Data loaded" once the data has been loaded

Start with `.shape` and `.info()`.

In [23]:
# Start with `.shape` and `.info()`
### YOUR CODE HERE ###
data = taxi_data.copy() 

data.shape

data.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22699 entries, 0 to 22698
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             22699 non-null  int64  
 1   VendorID               22699 non-null  int64  
 2   tpep_pickup_datetime   22699 non-null  object 
 3   tpep_dropoff_datetime  22699 non-null  object 
 4   passenger_count        22699 non-null  int64  
 5   trip_distance          22699 non-null  float64
 6   RatecodeID             22699 non-null  int64  
 7   store_and_fwd_flag     22699 non-null  object 
 8   PULocationID           22699 non-null  int64  
 9   DOLocationID           22699 non-null  int64  
 10  payment_type           22699 non-null  int64  
 11  fare_amount            22699 non-null  float64
 12  extra                  22699 non-null  float64
 13  mta_tax                22699 non-null  float64
 14  tip_amount             22699 non-null  float64
 15  to

Use `.head()`.

In [24]:
# Use `.head()`
data.head(n = 10)
### YOUR CODE HERE ###



,Unnamed: 0,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,24870114,2,03/25/2017 8:55:43 AM,03/25/2017 9:09:47 AM,6,3.34,1,N,100,231,1,13.0,0.0,0.5,2.76,0.0,0.3,16.56
1,35634249,1,04/11/2017 2:53:28 PM,04/11/2017 3:19:58 PM,1,1.80,1,N,186,43,1,16.0,0.0,0.5,4.00,0.0,0.3,20.80
2,106203690,1,12/15/2017 7:26:56 AM,12/15/2017 7:34:08 AM,1,1.00,1,N,262,236,1,6.5,0.0,0.5,1.45,0.0,0.3,8.75
3,38942136,2,05/07/2017 1:17:59 PM,05/07/2017 1:48:14 PM,1,3.70,1,N,188,97,1,20.5,0.0,0.5,6.39,0.0,0.3,27.69
4,30841670,2,04/15/2017 11:32:20 PM,04/15/2017 11:49:03 PM,1,4.37,1,N,4,112,2,16.5,0.5,0.5,0.00,0.0,0.3,17.80
5,23345809,2,03/25/2017 8:34:11 PM,03/25/2017 8:42:11 PM,6,2.30,1,N,161,236,1,9.0,0.5,0.5,2.06,0.0,0.3,12.36
6,37660487,2,05/03/2017 7:04:09 PM,05/03/2017 8:03:47 PM,1,12.83,1,N,79,241,1,47.5,1.0,0.5,9.86,0.0,0.3,59.16
7,69059411,2,08/15/2017 5:41:06 PM,08/15/2017 6:03:05 PM,1,2.98,1,N,237,114,1,16.0,1.0,0.5,1.78,0.0,0.3,19.58
8,8433159,2,02/04/2017 4:17:07 PM,02/04/2017 4:29:14 PM,1,1.20,1,N,234,249,2,9.0,0.0,0.5,0.00,0.0,0.3,9.80
9,95294817,1,11/10/2017 3:20:29 PM,11/10/2017 3:40:55 PM,1,1.60,1,N,239,237,1,13.0,0.0,0.5,2.75,0.0,0.3,16.55


Create `trip_duration`.

In [25]:
#convert tpep_dropoff_datetime and tpep_pickup_datetime from to datetime format
data['drop_off_converted'] = pd.to_datetime(data['tpep_dropoff_datetime'], format="%m/%d/%Y %I:%M:%S %p" )

data['pick_up_converted'] = pd.to_datetime(data['tpep_pickup_datetime'], format="%m/%d/%Y %I:%M:%S %p" )
data['trip_duration'] = (data['drop_off_converted'] - data['pick_up_converted']) / np.timedelta64(1, 'm')

data.head(n = 5)



,Unnamed: 0,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,...,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,drop_off_converted,pick_up_converted,trip_duration
0,24870114,2,03/25/2017 8:55:43 AM,03/25/2017 9:09:47 AM,6,3.34,1,N,100,231,...,13.0,0.0,0.5,2.76,0.0,0.3,16.56,2017-03-25 09:09:47,2017-03-25 08:55:43,14.066667
1,35634249,1,04/11/2017 2:53:28 PM,04/11/2017 3:19:58 PM,1,1.80,1,N,186,43,...,16.0,0.0,0.5,4.00,0.0,0.3,20.80,2017-04-11 15:19:58,2017-04-11 14:53:28,26.500000
2,106203690,1,12/15/2017 7:26:56 AM,12/15/2017 7:34:08 AM,1,1.00,1,N,262,236,...,6.5,0.0,0.5,1.45,0.0,0.3,8.75,2017-12-15 07:34:08,2017-12-15 07:26:56,7.200000
3,38942136,2,05/07/2017 1:17:59 PM,05/07/2017 1:48:14 PM,1,3.70,1,N,188,97,...,20.5,0.0,0.5,6.39,0.0,0.3,27.69,2017-05-07 13:48:14,2017-05-07 13:17:59,30.250000
4,30841670,2,04/15/2017 11:32:20 PM,04/15/2017 11:49:03 PM,1,4.37,1,N,4,112,...,16.5,0.5,0.5,0.00,0.0,0.3,17.80,2017-04-15 23:49:03,2017-04-15 23:32:20,16.716667


Check for missing data and duplicates using `.isna()` and `.drop_duplicates()`.

In [31]:
### YOUR CODE HERE ###
data.shape
# display shape of duplicate 
print(f"Shape of data with duplicated dropped: {data.drop_duplicates().shape}")

#display total of missing value
print(f"Total count of missing values {data.isna().sum().sum()}")

#display missing value per column in data
data.isna().sum()




Shape of data with duplicated dropped: (22699, 21)
Total count of missing values 0


Unnamed: 0               0
VendorID                 0
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
passenger_count          0
trip_distance            0
RatecodeID               0
store_and_fwd_flag       0
PULocationID             0
DOLocationID             0
payment_type             0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
total_amount             0
drop_off_converted       0
pick_up_converted        0
trip_duration            0
dtype: int64

Use `.describe()`.

In [32]:
# Display Statistics with A/B Testing
data.describe(include= 'all')




,Unnamed: 0,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,...,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,drop_off_converted,pick_up_converted,trip_duration
count,2.269900e+04,22699.000000,22699,22699,22699.000000,22699.000000,22699.000000,22699,22699.000000,22699.000000,...,22699.000000,22699.000000,22699.000000,22699.000000,22699.000000,22699.000000,22699.000000,22699,22699,22699.000000
unique,NaN,NaN,22687,22688,NaN,NaN,NaN,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,07/03/2017 3:45:19 PM,10/18/2017 8:07:45 PM,NaN,NaN,NaN,N,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,2,2,NaN,NaN,NaN,22600,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,5.675849e+07,1.556236,NaN,NaN,1.642319,2.913313,1.043394,NaN,162.412353,161.527997,...,13.026629,0.333275,0.497445,1.835781,0.312542,0.299551,16.310502,2017-06-29 07:49:49.799726848,2017-06-29 07:32:48.973126656,17.013777
min,1.212700e+04,1.000000,NaN,NaN,0.000000,0.000000,1.000000,NaN,1.000000,1.000000,...,-120.000000,-1.000000,-0.500000,0.000000,0.000000,-0.300000,-120.300000,2017-01-01 00:17:20,2017-01-01 00:08:25,-16.983333
25%,2.852056e+07,1.000000,NaN,NaN,1.000000,0.990000,1.000000,NaN,114.000000,112.000000,...,6.500000,0.000000,0.500000,0.000000,0.000000,0.300000,8.750000,2017-03-30 03:11:20.500000,2017-03-30 03:09:38.500000,6.650000
50%,5.673150e+07,2.000000,NaN,NaN,1.000000,1.610000,1.000000,NaN,162.000000,162.000000,...,9.500000,0.000000,0.500000,1.350000,0.000000,0.300000,11.800000,2017-06-23 12:55:11,2017-06-23 12:35:57,11.183333
75%,8.537452e+07,2.000000,NaN,NaN,2.000000,3.060000,1.000000,NaN,233.000000,233.000000,...,14.500000,0.500000,0.500000,2.450000,0.000000,0.300000,17.800000,2017-10-02 10:53:47,2017-10-02 10:34:34,18.383333
max,1.134863e+08,2.000000,NaN,NaN,6.000000,33.960000,99.000000,NaN,265.000000,265.000000,...,999.990000,4.500000,0.500000,200.000000,19.100000,0.300000,1200.290000,2017-12-31 23:49:24,2017-12-31 23:45:30,1439.550000


Create a scatterplot to visualize the relationship between variables of interest.

In [ ]:
# Create a scatterplot to visualize the relationship between variables of interest
### YOUR CODE HERE ###



Create a pairplot to visualize pairwise relationships between relevant variables.

In [ ]:
# Create a pairplot to visualize pairwise relationships between variables in the data
### YOUR CODE HERE ###



### Task 2b. Address any outliers

Use a boxplot to visualize any outliers.

In [ ]:
# Create boxplot to visualize the outliers
### YOUR CODE HERE ###



Use a boxplot to visualize the distribution of the data without outliers.

In [ ]:
# Create boxplot to visualize distribution of data without outliers
### YOUR CODE HERE ###



Remove outliers as needed.

In [ ]:
# Remove outliers as needed
### YOUR CODE HERE ###



### Task 2c. Identify correlations

Next, code a correlation matrix to help determine most correlated variables.

In [ ]:
# Correlation matrix to help determine most correlated variables
### YOUR CODE HERE ###



Visualize a correlation heatmap of the data.

In [ ]:
# Create correlation heatmap
### YOUR CODE HERE ###



### Task 3a. Select outcome variable and features

Set your Y and X variables. Y represents the outcome variable, and X represents the features.

In [ ]:
# Set your Y and X variables
### YOUR CODE HERE ###



### Task 3b. Pre-process data

To help with processing time, consider dropping irrelevant and redundant columns.

In [ ]:
# Pre-process data
### YOUR CODE HERE ###



Use `StandardScaler()` and `fit_transform()` to standardize the X variables.

In [ ]:
# Standardize the X variables
### YOUR CODE HERE ###



### Task 3c. Build model

Create training and testing sets.

In [ ]:
# Create training and testing sets
#### YOUR CODE HERE ####


Build and fit your model to the data.

In [ ]:
# Build and fit your model to the data
### YOUR CODE HERE ###



### Task 3d. Evaluate model

Evaluate your model performance by calculating the residual sum of squares and the explained variance score (R^2). Calculate the Mean Absolute Error, Mean Squared Error, and the Root Mean Squared Error.

In [ ]:
# Evaluate the model performance on the training data
### YOUR CODE HERE ###



In [ ]:
# Evaluate the model performance on the testing data
### YOUR CODE HERE ###



### Task 4a. Results

If the linear regression assumptions are met, the model results can be appropriately interpreted.

Use the code cell below to get `actual`,`predicted`, and `residual` for the testing set, and store them as columns in a `results` dataframe.

In [ ]:
# Create a `results` dataframe
### YOUR CODE HERE ###



### Task 4b. Visualize model results

Create a scatterplot to visualize `predicted` over `actual`.

In [ ]:
# Create a scatterplot to visualize `predicted` over `actual`
### YOUR CODE HERE ###



Visualize the distribution of the `residuals`.

In [ ]:
# Visualize the distribution of the `residuals`
### YOUR CODE HERE ###



Create a scatterplot of `residuals` over `predicted`.

In [ ]:
# Create a scatterplot of `residuals` over `predicted`
### YOUR CODE HERE ###

